In [2]:
import pandas as pd

In [39]:
raw_rank_df = pd.read_csv('/Volumes/GoogleDrive/My Drive/AmzJet - ZonTracker/Troubleshooting/GeoRank/Interpolation Crashes/data.csv')
raw_rank_df['date'] = pd.to_datetime(raw_rank_df['date'])
raw_rank_df.head(3)

,Unnamed: 0,date,state,rank,avail_msg,get_by_date
0,0,2019-10-24,DE,136,NaN,2019-10-25
1,1,2019-10-24,MA,103,NaN,2019-10-25
2,2,2019-10-24,NY,136,NaN,2019-10-25


In [40]:
daily_rank_df = raw_rank_df.groupby(['state', 'date']).agg({
    'rank':
    'mean',
    'avail_msg':
    'last',
    'get_by_date':
    'last'
})
rank_with_state_cols = daily_rank_df.unstack('state')['rank']
rank_with_state_cols

state,AK,AL,AR,AZ,CA,CO,CT,DE,FL,GA,...,SC,SD,TN,TX,VA,VT,WA,WI,WV,WY
date,,,,,,,,,,,,,,,,,,,,,
2019-10-24,NaN,136.0,NaN,NaN,NaN,NaN,136.0,136.0,NaN,NaN,...,136.0,136.0,137.0,NaN,136.0,NaN,NaN,136.0,NaN,NaN
2019-10-27,NaN,NaN,NaN,NaN,NaN,NaN,128.0,129.0,128.0,NaN,...,NaN,NaN,NaN,128.0,129.0,NaN,128.0,129.0,NaN,109.5
2019-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-01,NaN,122.0,78.0,NaN,NaN,NaN,NaN,122.0,NaN,NaN,...,NaN,NaN,122.0,NaN,122.0,NaN,NaN,NaN,122.0,98.0
2019-11-04,NaN,88.0,NaN,101.0,NaN,88.0,NaN,NaN,88.0,NaN,...,NaN,101.0,NaN,88.0,127.0,NaN,118.0,NaN,NaN,NaN
2019-11-05,NaN,NaN,83.0,89.0,119.0,81.0,103.0,NaN,NaN,NaN,...,112.0,NaN,NaN,NaN,NaN,NaN,130.0,NaN,NaN,57.0
2019-11-06,79.0,107.0,NaN,83.0,84.0,NaN,72.0,65.0,NaN,102.0,...,NaN,81.0,83.0,NaN,NaN,88.0,112.0,NaN,73.0,NaN
2019-11-07,NaN,NaN,NaN,36.0,NaN,NaN,49.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
rank_with_state_cols = rank_with_state_cols.interpolate(
                    method='spline',
                    order=1,
                    axis=0,
                    limit=1000,
                    limit_area='inside')


error: (m>k) failed for hidden m: fpcurf0:m=1

In [41]:
for column, values in rank_with_state_cols.items():
    if len(values[pd.notnull(values)]) >= 2:
        x = values.interpolate(
                            method='spline',
                            order=1,
                            axis=0,
                            limit=1000,
                            limit_area='inside')
        rank_with_state_cols[column]= x
rank_with_state_cols      
# rank_with_state_cols = rank_with_state_cols.interpolate(
#                     method='spline',
#                     order=1,
#                     axis=0,
#                     limit=1000,
#                     limit_area='inside')


state,AK,AL,AR,AZ,CA,CO,CT,DE,FL,GA,...,SC,SD,TN,TX,VA,VT,WA,WI,WV,WY
date,,,,,,,,,,,,,,,,,,,,,
2019-10-24,NaN,136.000000,NaN,NaN,NaN,NaN,136.000000,136.000000,NaN,NaN,...,136.0,136.000000,137.000000,NaN,136.000000,NaN,NaN,136.0,NaN,NaN
2019-10-27,NaN,130.477563,NaN,NaN,NaN,NaN,128.000000,129.000000,128.000000,NaN,...,130.0,126.243356,131.186292,128.0,129.000000,NaN,128.000000,129.0,NaN,109.500000
2019-10-31,NaN,122.990231,NaN,NaN,NaN,NaN,116.047626,122.376909,83.000000,NaN,...,122.0,112.965152,122.715796,108.0,124.171663,NaN,123.472343,NaN,NaN,99.296434
2019-11-01,NaN,122.000000,78.00,NaN,NaN,NaN,113.145284,122.000000,85.133895,NaN,...,120.0,109.645602,122.000000,103.0,122.000000,NaN,122.349147,NaN,122.0,98.000000
2019-11-04,NaN,88.000000,81.75,101.0,NaN,88.0,104.438260,87.608376,88.000000,NaN,...,114.0,101.000000,98.556867,88.0,127.000000,NaN,118.000000,NaN,92.6,67.485211
2019-11-05,NaN,97.894270,83.00,89.0,119.0,81.0,103.000000,76.657612,NaN,NaN,...,112.0,90.898996,91.209765,NaN,NaN,NaN,130.000000,NaN,82.8,57.000000
2019-11-06,79.0,107.000000,NaN,83.0,84.0,NaN,72.000000,65.000000,NaN,102.0,...,NaN,81.000000,83.000000,NaN,NaN,88.0,112.000000,NaN,73.0,NaN
2019-11-07,NaN,NaN,NaN,36.0,NaN,NaN,49.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
